In [85]:
import logging

from google.cloud import storage

client = storage.Client.from_service_account_json('service/mktg-test-cloud-storage.json')

bucket = client.get_bucket('mktg-data-bucket-1')



In [29]:
from pymongo import MongoClient

# establish mongoDB connection
mongo_con='mongodb://adminProd:{}@planningtool-shard-00-00-k7wis.mongodb.net:27017,planningtool-shard-00-01-k7wis.mongodb.net:27017,planningtool-shard-00-02-k7wis.mongodb.net:27017/test?ssl=true&replicaSet=PlanningTool-shard-0&authSource=admin&w=majority'
mongo_password='Iu25kjB7Hyke0dS1'
mongo_env='production'

mongodb = MongoClient(mongo_con.format(mongo_password))[mongo_env]

channels = dict()
for c in list(mongodb.channels.find({'active':True}, {'prettyName':1})):
    channels[c['_id']] = c['prettyName']
    
regions = dict()
for r in list(mongodb.divisions.find({'active':True}, {'prettyName':1})):
    regions[r['_id']] = r['prettyName']
    
productGroups = dict()
for p in list(mongodb.productgroups.find({'active':True}, {'prettyName':1})):
    productGroups[p['_id']] = p['prettyName']
    
markets = dict()
states = dict()
market_regions = dict()
market_productgroups = dict()

for m in list(mongodb.markets.find({'active':True}, {'prettyName':1, 'state':1, 
                                                     'divisionId':1, 'productGroupId':1})):
    markets[m['_id']] = m['prettyName']
    states[m['_id']] = m['state']
    market_regions[m['_id']] = regions[m['divisionId']]
    market_productgroups[m['_id']] = productGroups[m['productGroupId']]

departments = dict()
for d in list(mongodb.departments.find({'active':True}, {'prettyName':1})):
    departments[d['_id']] = d['prettyName']

brands = dict()
for b in list(mongodb.brands.find({'active':True}, {'prettyName':1})):
    brands[b['_id']] = b['prettyName']

fiscalYears = dict()
for f in list(mongodb.fiscalyearplans.find({'active':True}, {'fiscalYear':1})):
    fiscalYears[f['_id']] = f['fiscalYear']
    
programs = []
for x in mongodb.programs.aggregate([
            {'$unwind': '$monthItems'},
            {'$match': {
                'monthItems.targetCount': {'$gt': 0},
                'fiscalYearPlanId':{'$in':list(fiscalYears.keys())},
                'active': True,
                }
            },
            {'$project': {
                '_id': 0,
                'fiscalYearPlanId': 1,
                'marketId': 1,
                'channelId': 1,
                'departmentId':1,
                'prettyName': 1,
                'month': '$monthItems.monthNumber',
                'monthName': '$monthItems.prettyName',
                'count': '$monthItems.targetCount',
                'cost': '$monthItems.eventCost',
                'kpi':'$monthItems.kpi'
            }
            }
        ]):
    p = dict()
    p['fiscal'] = fiscalYears[x['fiscalYearPlanId']]
    p['productgroup'] = market_productgroups[x['marketId']]
    p['market'] = markets[x['marketId']]
    p['state'] = states[x['marketId']]
    p['region'] = market_regions[x['marketId']]
    p['department'] = departments[x['departmentId']]
    p['channel'] = channels[x['channelId']]
    p['month'] = x['month']
    if x['month'] >= 10:
        p['quarter'] = 2
    elif x['month']>=7:
        p['quarter'] = 1
    elif x['month']>=4:
        p['quarter'] = 4
    elif x['month']>=1:
        p['quarter'] = 3
        
    p['event'] = x['count']
    p['cost'] = x['cost']
    p['kpi'] = x['kpi']
    
    programs.append(p)
    
df = pd.DataFrame(programs)

df

,fiscal,productgroup,market,state,region,department,channel,month,quarter,event,cost,kpi
0,FY20,Spirits,Missouri,MO,Central,National Accounts,Off Premise,3,3,2,0.00,0.0
1,FY20,Spirits,Missouri,MO,Central,National Accounts,Off Premise,4,4,3,0.00,0.0
2,FY20,Spirits,Colorado,CO,West,National Accounts,Off Premise,3,3,7,0.00,0.0
3,FY20,Spirits,Colorado,CO,West,National Accounts,Off Premise,4,4,8,0.00,0.0
4,FY20,Spirits,Wisconsin,WI,Central,National Accounts,Off Premise,3,3,25,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
33261,FY21 Q1,Spirits,New Jersey,NJ,East,Central Sampling,Off Premise,8,1,1,250.90,0.0
33262,FY21 Q1,Spirits,New Jersey,NJ,East,Central Sampling,Off Premise,9,1,1,250.90,0.0
33263,FY21 Q1,Spirits,Nevada,NV,East,Central Sampling,Off Premise,9,1,8,361.47,0.0
33264,FY21 Q1,Spirits,Colorado,CO,West,Central Sampling,Off Premise,9,1,3,210.73,0.0


In [96]:
import datetime as dt
import pandas as pd
from io import StringIO # if going with no saving csv file

# filename = 'planning-{}.csv'.format(str(dt.date.today()))
filename = 'planning.csv'

f = StringIO()
df.to_csv(f, index=False)
f.seek(0)

bucket.blob(filename).upload_from_file(f, content_type='text/csv')

In [94]:
import json
from bigquery_schema_generator.generate_schema import SchemaGenerator

f = StringIO()

generator = SchemaGenerator()
check = df.to_dict('records')[0]

schema = dict()
schema_string = ''
count = 0
for k in check:
    schema_string += '{}:{}'.format(k, generator.infer_bigquery_type(check[k])[1])
    if count < len(check)-1:
        schema_string += ','
    schema[k] = generator.infer_bigquery_type(check[k])[1]
    count+=1

types = dict(df.dtypes)

json.dump(schema, f)

f.seek(0)

schema_filename = 'planning_schema.json'
bucket.blob(schema_filename).upload_from_file(f, content_type='application/json')

In [95]:
schema_string

'fiscal:STRING,productgroup:STRING,market:STRING,state:STRING,region:STRING,department:STRING,channel:STRING,month:INTEGER,quarter:INTEGER,event:INTEGER,cost:FLOAT,kpi:FLOAT'